#### <center> At scale: Parallelization on your local computers

In [1]:
library(parallel)
for (i in seq(2,16,2)) {
  cl <- makeCluster(i)
  ptm <- proc.time()
  parLapply(cl, 1:10000000,
            function(exponent){
                2^exponent
            })
  run.time <- proc.time() - ptm
  stopCluster(cl)
  print (run.time)
}

   user  system elapsed 
 12.938   0.462  31.686 
   user  system elapsed 
  8.402   0.325  17.984 
   user  system elapsed 
  8.738   0.296  15.045 
   user  system elapsed 
  8.512   0.225  12.948 
   user  system elapsed 
  7.988   0.232  11.984 
   user  system elapsed 
  9.164   0.221  12.456 
   user  system elapsed 
  8.180   0.296  10.949 
   user  system elapsed 
  8.226   0.235  10.729 


#### <center> At scale: Big Data analytics on Spark

In [5]:
#if (nchar(Sys.getenv("SPARK_HOME")) < 1) {
#  Sys.setenv(SPARK_HOME = "/usr/hdp/current/spark-client")
#}

spark_submit_args <- paste("--master yarn-client",
                           "--executor-memory 32g",
                           "--num-executors 4",
                           "--executor-cores 10",
                           paste("--conf spark.executorEnv.PATH=",
                                 paste(Sys.getenv("PATH"),
                                       "/usr/local/share/jupyterhub/env/R/lib/R/bin/",sep=":"),
                                 sep=""),
                           "sparkr-shell",sep=" ")

Sys.setenv("SPARKR_SUBMIT_ARGS"=spark_submit_args)

In [6]:
library(SparkR)
sc <- sparkR.init(sparkPackages="com.databricks:spark-csv_2.11:1.2.0")
sqlContext <- sparkRSQL.init(sc)

Launching java with spark-submit command spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 --master yarn-client --executor-memory 32g --num-executors 4 --executor-cores 10 --conf spark.executorEnv.PATH=/software/hdp/6.0:/software/singularity/2.2/bin:/software/hdf5/1.10.0/bin:/software/openmpi/1.10.3_gcc610/bin:/software/gcc/5.3.0/bin:/software/zeromq/4.1.5/bin:/software/matlab/R2015a/toolbox/distcomp/bin:/software/matlab/R2015a/bin/glnxa64:/software/matlab/R2015a/bin:/software/anaconda3/4.2.0/bin:/usr/lib64/qt-3.3/bin:/bin:/usr/bin:/usr/local/bin:/usr/local/sbin:/usr/sbin:/sbin:/opt/ibutils/bin:/opt/mx/bin:/opt/pbs/default/bin:/opt/pbs/default/sbin:/home/lngo/bin:/usr/local/share/jupyterhub/env/R/lib/R/bin/ sparkr-shell /local_scratch/pbs.9015728.pbs02/RtmpF5g7Ra/backend_port32385272a3c7 


In [7]:
ptm <- proc.time()
reddit <- read.json(sqlContext, "hdfs:///datasets/reddit_data/*/*.bz2")
proc.time() - ptm

    user   system  elapsed 
   0.008    0.000 3628.151 

In [8]:
print (printSchema(reddit))

root
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- mod_reports: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- name: string (nullable = true)
 |-- num_reports: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- r

In [9]:
ptm <- proc.time()
count (reddit)
proc.time() - ptm

[1] 1659361605

    user   system  elapsed 
   0.010    0.000 1889.631 

In [10]:
sparkR.stop()